In [1]:
import re
import parse
import pdfplumber
import pandas as pd
import numpy as np
from collections import namedtuple
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.sample import sample_config
from tableau_api_lib.utils import querying
from tableau_api_lib.utils.common import flatten_dict_column
import os
import getpass
import gspread
from string import digits

Lineforcsv = namedtuple('Line', 'Invoice LSP Function JobNo Code Charge')



Tableau

In [93]:
LOGIN = getpass.getpass(prompt='Password: ', stream=None)


tableau_server_config = {
        'my_env': {
                'server': 'https://us-east-1.online.tableau.com',
                'api_version': '3.16',
                'username': 'greg_di_lella@qintl.com',
                'password': LOGIN,
                'site_name': 'q',
                'site_url': 'q'
        }
}
conn = TableauServerConnection(tableau_server_config, env='my_env')
response = conn.sign_in()

site_views_df = querying.get_views_dataframe(conn)
site_views_detailed_df = flatten_dict_column(site_views_df, keys=['name', 'id'], col_name='workbook')
ATS_data = conn.query_view_data(view_id='2cf7d6bb-53d8-4af1-be36-282498463afe')

with open("ATS_data.csv","wb") as file:
    file.write(ATS_data.content)
    
LSP_list_Tableau = pd.read_csv('ATS_data.csv',encoding= 'unicode_escape',low_memory=False)

LSP_list_Tableau.rename(columns = {'Job No':'JobNo'}, inplace = True)
LSP_list_Tableau['Vendor'].replace('FRA13','FRA06',inplace=True)
print(response)


Password: ········
<Response [200]>


Class Definitions

In [87]:
Flag_re = re.compile(r'(.*) (\d+,\d{3}.\d{2}|\d+.\d{2}) EUR')

comp = Flag_re.search(r'Total excl. VAT 654.00 EUR')
comp.group(2)

'654.00'

In [126]:
class LSP_CLASS: 
          
    LSP_re = re.compile(r'Factura/Invoice|SAMEDAY|in time|ODYSSEY|Stat Experts|The Courier Company|CANADA WORLDWIDE|IDQUIINTJFK|IDSTEAVIJFK|Special Dispatch|Time Couriers|Attachement to invoice|BUFFALO EXPRESS|Atlas Express|ACTIVE TRANSPORT|Courier Corporation Of Hawaii|Schneider|Collective Invoice')
    DICT = {'Factura/Invoice':'MAD07','SAMEDAY':'SEA70','in time':'FRA06','ODYSSEY':'BRU09','Stat Experts':'IAD58','The Courier Company':'CVG02','CANADA WORLDWIDE':'YYZ37','IDQUIINTJFK':'CTF','IDSTEAVIJFK':'CTF','Special Dispatch':'IND12','Time Couriers':'DEN42','Attachement to invoice':'MXP06','BUFFALO EXPRESS':'BUF21','Atlas Express':'DCA48','ACTIVE TRANSPORT':'SEL07','Courier Corporation Of Hawaii':'HNL09','Schneider':'ZRH00','Collective Invoice':'AMS14'}  
    
    MAD_Job_re = re.compile(r'HAWB: (\d{7,9}\w{1,})')
    MAD_line_re = re.compile(r'(.*) (\d+,\d{2})$')
    MAD_Flag_re = re.compile(r'\d+,\d{2} Euros$|\d+\.\d+,\d{2} Euros$')

    SEA_Job_re = re.compile(r'REFER: (\d{7,9}\w{1,})')
    SEA_line_re = re.compile(r'(.*) \d+.\d+ 0.00 (\d+.\d+)')
    SEA_Flag_re = re.compile(r'^(\d+.\d{2}) USD$')
    SEA_LSP_loca_re = re.compile(r', NJ|, WA')

    FRA_Job_re = re.compile(r'Reference:\s*(\d{7,9}\w{1,})')
    FRA_line_re = re.compile(r'(.*) (.*) EUR')
    FRA_Flag_re = re.compile(r'Total Amount (.*) EUR|Total amount (.*) EUR')

    BRU_Job_re = re.compile(r'(\d{2}/\d{2}/\d{4}) ([a-zA-Z0-9]{15})')
    BRU_line_re = re.compile(r'[A-Z] [A-Z]{3} (.*) (\S*) 0R')
    BRU_Flag_re = re.compile(r'^TOTAL (.*) EUR$')
    
    IAD_Job_re = re.compile(r'Ref#:(\d{7,9}\w{1,})')
    IAD_line_re = re.compile(r'(\w+)\s*:\s+(\d+\.\d{2})')
    IAD_line2_re = re.compile(r'(\w+)\s*:\s+(\.\d{2})')
    IAD_start_re = re.compile(r'(\d+/\d+/\d{2})')
    IAD_Flag_re = re.compile(r'11133 .*(\d{2,3},\d{3}.\d{2})')
    
    CVG_Job_re = re.compile(r'Job#:\s*(\d{7,9}\w{1,})')
    CVG_line1_re = re.compile(r'(\d+\.\d{2}) (\d+\.\d{2}) (.*) \$')
    CVG_line2_re = re.compile(r'(\d+\.\d{2}) (.*)') 
    CVG_Flag_re = re.compile(r'Invoice Total \$(.*)')

    YYZ_Job_re = re.compile(r'Reference # (\d{7,9}\w{1,})')
    YYZ_line_re = re.compile(r'(.*) (\d+.\d{2})$')
    YYZ_Flag_re = re.compile(r'Grand Total: USD \$ (\d+.\d{2})$')
    
    CTF_Job_re = re.compile(r' (\d{7,9}[A-Za-z]{1,})')
    CTF_line_re = re.compile(r'(.* \d*\.\d{2}$|.* \d+,\d+\.\d{2}$)')
    CTF_Flag_re = re.compile(r'TOTAL AUD (.*)')
    CTF_Invoice_re = re.compile('(INVOICE \d{8})')
    CTF_AUSYD_re = re.compile('AUSYD')
    CTF_AUPER_re = re.compile('AUPER')
    CTF_AUMEL_re = re.compile('AUMEL')
    CTF_AUADL_re = re.compile('AUADL')
    CTF_AUBNE_re = re.compile('AUBNE')
    
    IND_Job_re = re.compile(r'(\d{7,9}\w{1,})')
    IND_line_re = re.compile(r'(.*) \$(\d+.\d{2}$)')
    IND_line2_re = re.compile(r'(.*) \$(\d+,\d+.\d{2}$)')
    IND_Flag_re = re.compile(r'^\d{5} \$(.*)')
    
    DEN_Job_re = re.compile(r'\d{6}\s+(\d{7,9}\w{1})')
    DEN_line1_re = re.compile(r'(\d+.\d{2})\s+\d+.\d{2}$')
    DEN_line2_re = re.compile(r'(\d+.\d \d)\s+(\d+.\d{2}) ([A-Z0-9]{2})\s+\d+.\d{2}$')
    DEN_line3_re = re.compile(r'(\d+.\d \d)\s+(\d+.\d{2})\s+([A-Z]\s+[A-Z0-9])\s+(\d+.\d{2})\s+([A-Z]\s+[A-Z0-9])\s+\d+.\d{2}$')
    DEN_line4_re = re.compile(r' (\d+.\d{2}) ([A-Z0-9]{2}$)')
    DEN_Flag_re = re.compile(r'Invoice\s+Total\s+(\d+.\d{2})$')
    
    MXP_Job_re = re.compile(r'\d{2}/\d{2}/\d{4} (\d{7,9}\w{1,})')
    MXP_line_re = re.compile(r'(.*) (\d+.\d{2}).*TER\)$')
    MXP_Flag_re = re.compile(r'Total shipment (.*)$')
    
    BUF_Job_re = re.compile(r'JOB (\d{7,9}\w{1,})$')
    BUF_line_re = re.compile(r'(.*) \d .* (\d+.\d{2})$')
    BUF_Flag_re = re.compile(r'BALANCE DUE \$(.*)$')
    
    DCA_Job_re = re.compile(r'Tracking number: (\d{7,9}\w{1,})')
    DCA_line_re = re.compile(r'\$(.*?)- (.*)')
    DCA_line2_re = re.compile(r'Base price: \$(.*)$')
    DCA_Flag_re = re.compile(r'Total due: \$(.*)$')
    
    SEL_Job_re = re.compile(r'JOB NO.:(\d{7,9}\w{1,})')
    SEL_line_re = re.compile(r'^([A-Z]{3}) .* (\d.*$)')
    SEL_Flag_re = re.compile(r'TOTAL  BALANCE DEBIT  USD  (.*)$')
    
    HNL_Job_re = re.compile(r'Reference number: (\d{7,9}\w{1,})')
    HNL_line_re = re.compile(r'\$(.*?)- (.*)')
    HNL_line2_re = re.compile(r'Base price: \$(.*)$')
    HNL_Flag_re = re.compile(r'Total due: \$(.*)$')
    
    ZRH_Job_re = re.compile(r'Reference: (\d{7,9}\w{1,})')
    ZRH_line_re = re.compile(r'^\d{3} (.*) 0% (.*)')
    ZRH_Flag_re = re.compile(r'Total amount in CHF (.*)$')
    
    AMS_Job_re = re.compile(r'Reference: (\d{7,9}\w{1,})')
    AMS_line_re = re.compile(r'(.*) (\d+,\d{3}.\d{2}|\d+.\d{2}) EUR')
    AMS_Flag_re = re.compile(r'Payment within 14 days Total amount (.*) EUR')       


    
#  --------------------------------------------------------------------------------------------   
    def MAD07():
        global Status
        global lines
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        
        
        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()


                for line in text.split('\n'):
#                     print(line)                    
                    comp1 = LSP_CLASS.MAD_Job_re.search(line)
                    comp3 = LSP_CLASS.MAD_line_re.search(line)
                    Flag = LSP_CLASS.MAD_Flag_re.search(line)

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': LSP, 'Invoice_Total': Flag.group()},ignore_index=True)

                    if comp1:
                        JOB = comp1.group(1).upper()

                    if comp3:
                        Charge = comp3.group(2)
                        Code = comp3.group(1)

#                         lines.append(Lineforcsv(file,'MAD07', 'P', JOB, Code, Charge))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'MAD07', 'Function': 'P', 'JobNo':JOB, 'Code':Code, 'Charge':Charge},ignore_index=True)
        
        
        pre_df['Charge'] = pre_df['Charge'].str.replace(',', '.')
        pre_df['Charge'] = pre_df['Charge'].astype(float)
        pre_df['Code'] = pre_df['Code'].str.replace(':','')
        pre_df['Code'] = pre_df['Code'].str.rstrip(' ')
        pre_df['Code'] = pre_df['Code'].str.upper()        
#         pre_df['Code'] = np.where(pre_df['Code'].str.contains('HAWB'), 'HAWB', pre_df['Code'])
        
        df = df.append(pre_df)

    def SEA70():
        global Status
        global lines
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        
        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)

                    comp1 = LSP_CLASS.SEA_Job_re.search(line)
                    comp3 = LSP_CLASS.SEA_line_re.search(line)
                    Flag = LSP_CLASS.SEA_Flag_re.search(line)
                    LSP_loca = LSP_CLASS.SEA_LSP_loca_re.search(line)
                 

                    if line.startswith('ORDER'):
                        JOB = None
                        LSP_comp= None
                        Charge = None 
                        Code = None
                        T1 = line.split()
                        Order = T1[2]

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': LSP, 'Invoice_Total': Flag.group()},ignore_index=True)
                    
                    if LSP_loca:
                        LSP_comp = LSP_loca.group()
                        if LSP_comp == ', NJ':
                            LSP_comp = 'EWR42'
                        elif LSP_comp == ', WA':
                            LSP_comp = 'SEA70'                                

                    if comp1:
                        JOB = comp1.group(1).upper()
                        lines.append(Lineforcsv(file, LSP_comp, 'P', JOB, None, None))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': LSP_comp, 'Function': 'P', 'JobNo':JOB, 'Code':None, 'Charge':None},ignore_index=True)
                        

                    if comp3:
                        Charge = comp3.group(2)
                        Items = comp3.group(1).split()
                        try:
                            Code = Items[-2] +' '+ Items[-1]
                        except:
                            Code = Items[0]

                        lines.append(Lineforcsv(file, LSP_comp, 'P', JOB, Code, Charge))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': LSP_comp, 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge},ignore_index=True)
            
            pre_df['LSP'] = pre_df['LSP'].fillna(method="bfill")
            pre_df['JobNo'] = pre_df['JobNo'].fillna(method="bfill")
            pre_df = pre_df.dropna()
            pre_df['Charge'] = pre_df['Charge'].astype(float, errors = 'raise')
            pre_df['Code'] = pre_df['Code'].str.upper()
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('@'), 'PICKUP', pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('HOLD OVER'), 'HOLD OVER', pre_df['Code'])
            
            df = df.append(pre_df)
                        
    def FRA06():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
       
        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)
                    if line.startswith('Reference'):
                        JOB = None
                        Charge = None 
                        Code = None

                    comp1 = LSP_CLASS.FRA_Job_re.search(line)
                    comp3 = LSP_CLASS.FRA_line_re.search(line)
                    Flag = LSP_CLASS.FRA_Flag_re.search(line)


                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': LSP, 'Invoice_Total': Flag.group()},ignore_index=True)

                    if comp1:
                        JOB = comp1.group(1).upper()

                    if comp3:
                        Charge = comp3.group(2)
                        Items = comp3.group(1).split()

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'FRA06', 'Function': 'P', 'JobNo':JOB, 'Code':Items[0], 'Charge':Charge},ignore_index=True)
            
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','')
            pre_df['Charge'] = pre_df['Charge'].astype(float, errors = 'raise')
#             pre_df['Charge'] = pre_df['Charge'].map(lambda x:float(x) )
            pre_df = pre_df[pre_df['Code'].str.contains('Tax|Total')==False]
            
            df = df.append(pre_df)
                        
                        
    def BRU09():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        

        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):

                    comp = LSP_CLASS.BRU_Job_re.search(line)
                    comp2 = LSP_CLASS.BRU_line_re.search(line)
                    Flag = LSP_CLASS.BRU_Flag_re.search(line)
                    
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': LSP, 'Invoice_Total': Flag.group()},ignore_index=True)
#                         print(Status)
                    if comp:
                        PUD, JOB = comp.group(1), comp.group(2)

                    if comp2:
                        REF,Charge  = comp2.group(1), comp2.group(2)

#                         lines.append(Lineforcsv(file,'BRU09','P', JOB, REF, Charge))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'BRU09', 'Function': 'P', 'JobNo':JOB, 'Code':REF, 'Charge':Charge},ignore_index=True)

                    elif line.startswith('10,00 %'):
                        REF = 'TAX'
                        items = line.split()
#                         lines.append(Lineforcsv(file,'BRU09','P', JOB, REF, items[-4]))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'BRU09', 'Function': 'P', 'JobNo':JOB, 'Code':REF, 'Charge':items[-4]},ignore_index=True)
                  
                    elif line.startswith('FSS'):
                        REF = 'TAX'
                        items = line.split()
#                         lines.append(Lineforcsv(file,'BRU09','P', JOB, REF, items[-4]))
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'BRU09', 'Function': 'P', 'JobNo':JOB, 'Code':REF, 'Charge':items[-4]},ignore_index=True)
    
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('NEXT DROP'), 'NEXT DROP', pre_df['Code'])
        
            df = df.append(pre_df)
                        
    def IAD58():
        pre_Status = pd.DataFrame({'Invoice': [], 'LSP': [], 'Invoice_Total': []})
        global Status
        global lines
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code': [], 'Charge': []})
        
        
        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:

                text = page.extract_text()

                for line in text.split('\n'):

#                     print(line)
                    
                    if LSP_CLASS.IAD_start_re.search(line):
                        JOB = None
                        Charge1 = None 
                        Code = None

                    comp1 = LSP_CLASS.IAD_Job_re.search(line)
                    comp2 = LSP_CLASS.IAD_line_re.search(line)
                    comp3 = LSP_CLASS.IAD_line2_re.search(line)
                    Flag= LSP_CLASS.IAD_Flag_re.search(line)

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'IAD58', 'Invoice_Total': Flag.group(1)},ignore_index=True)
#                         print(Flag.group(1))
#                         print(line)

                    if comp1:
                        JOB = comp1.group(1).upper()
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'IAD58', 'Function': 'P', 'JobNo':JOB, 'Code':None, 'Charge':None},ignore_index=True)
                        

                    if comp2:
                        Code = comp2.group(1)
                        Charge = comp2.group(2)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'IAD58', 'Function': 'P', 'JobNo':JOB, 'Code':Code, 'Charge':Charge},ignore_index=True)
            
                    if comp3:
                        Code = comp3.group(1)
                        Charge = comp3.group(2)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'IAD58', 'Function': 'P', 'JobNo':JOB, 'Code':Code, 'Charge':Charge},ignore_index=True)
            

            pre_df['JobNo'] = pre_df['JobNo'].fillna(method="bfill")
            pre_df = pre_df.dropna()
            pre_df['Charge'] = pre_df['Charge'].map(lambda x:float(x))
            pre_df['Code'] = pre_df['Code'].replace("1",'ADD STOP')
            pre_df['Code'] = pre_df['Code'].str.upper()       
#             pre_Status = pre_Status.drop_duplicates(subset=['Invoice'], keep='last')
                                  
            df = df.append(pre_df)
            print(Status)
            
       
    def CVG02():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        

        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()
                for line in text.split('\n'):
#                     print(line)
                    if line.startswith('Job#:'):
                        JOB = None
                        Charge1 = None 
                        Charge2 = None
                        Charge3 = None
                        Code = None
                        Code2 = None

                    comp1 = LSP_CLASS.CVG_Job_re.search(line)
                    comp2 = LSP_CLASS.CVG_line1_re.search(line)
                    comp3 = LSP_CLASS.CVG_line2_re.search(line)
                    Flag = LSP_CLASS.CVG_Flag_re.search(line)

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'CVG02', 'Invoice_Total': Flag.group(1)},ignore_index=True)
#                         print(Status)

                    if comp1:
                        JOB = comp1.group(1).upper()

            
                    if comp2:
                        Charge1 = comp2.group(1)
                        Charge2 = comp2.group(2)
                        Code = comp2.group(3).upper()


                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'CVG02', 'Function': 'P', 'JobNo':JOB, 'Code':'BASE RATE', 'Charge':Charge1},ignore_index=True)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'CVG02', 'Function': 'P', 'JobNo':JOB, 'Code':Code, 'Charge':Charge2},ignore_index=True)


                    elif comp3: 
                        Charge3 = comp3.group(1)
                        Code2 = comp3.group(2).upper()


                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'CVG02', 'Function': 'P', 'JobNo':JOB, 'Code':Code2, 'Charge':Charge3},ignore_index=True)
            
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('\$')==True, 'BASE RATE', pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('WAIT'), 'WAIT TIME', pre_df['Code'])
            pre_df=pre_df.dropna()            
            
            df = df.append(pre_df)

                            
                        
    def YYZ37():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        JOB = None 
       
        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):

#                     print(line)
                    if line.startswith('SENDER'):
                        JOB = None
                        Charge = None 
                        Code = None

                    comp1 = LSP_CLASS.YYZ_Job_re.search(line)
                    comp3 = LSP_CLASS.YYZ_line_re.search(line)
                    Flag = LSP_CLASS.YYZ_Flag_re.search(line)
                    
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': LSP, 'Invoice_Total': Flag.group(1)},ignore_index=True)

                    if comp1:
                        JOB = comp1.group(1).upper()

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'YYZ37', 'Function': 'P', 'JobNo':JOB, 'Code':None, 'Charge':None},ignore_index=True)

                    if comp3:
                        Charge = comp3.group(2)
                        Code = comp3.group(1)


                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'YYZ37', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge},ignore_index=True)
                        

            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('\$')==True, None ,pre_df['Code'])
            pre_df['JobNo'] = pre_df['JobNo'].fillna(method="bfill")
            pre_df=pre_df.dropna()
            pre_df = pre_df.loc[pre_df['Code']!= 'None', :]
            

            df = df.append(pre_df)


        
    def YYZ_df_cleanup(x):
        global df
        df=x
        df=df.dropna()
#         df = df.loc[df['Code']!= 'None', :]
        df['Code'] = np.where(df['Code'].str.contains('INSIDE PICKUP'), 'ID',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('INSIDE DELIVERY'), 'ID',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('PICKUP'), 'PICKUP',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('VAN'), 'VAN',df['Code'])  
        df['Code'] = np.where(df['Code'].str.contains('FUEL SURCHARGE'), 'FUEL SURCHARGE',df['Code']) 
        df['Code'] = np.where(df['Code'].str.contains('AIRLINE DOCUMENTS'), 'AIRLINE DOCUMENTS',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('AIR FREIGHT'), 'AIR FREIGHT',df['Code'])  
        df['Code'] = np.where(df['Code'].str.contains('MILEAGE'), 'MILEAGE',df['Code'])         
        df['Code'] = np.where(df['Code'].str.contains('AFTER HOURS'), 'AFTER HOURS',df['Code'])        
        df['Code'] = np.where(df['Code'].str.contains('PICK UP'), 'PICKUP',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('LEECHES'), 'LEECHES',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('CUSTOMS'), 'CUSTOMS',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('2ND DRIVER'), '2ND DRIVER',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('BORDER CROSSING'), 'BORDER CROSSING',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('DRIVER PACK'), 'DRIVER PACK',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('DOWNLOAD DATA'), 'DOWNLOAD DATA',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('TERMINAL HANDLING'), 'TERMINAL HANDLING',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('PULL-A-PART'), 'PULL-A-PART',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('RE-MANIFEST'), 'RE-MANIFEST',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('WEIGHT CHARGE'), 'WEIGHT CHARGE',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('TAXES'), 'TAXES',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('RE PACK'), 'RE PACK',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('AIRLINE SCREENING'), 'AIRLINE SCREENING',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('AFTER HOUR'), 'AFTER HOURS',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('WAREHOUSING'), 'WAREHOUSE',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('SCREENING'), 'SCREENING',df['Code'])
        df['Code'] = np.where(df['Code'].str.contains('PALLETIZE'), 'PALLETIZE',df['Code'])
        
        
        
        
    def CT_Freight():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        JOB= None
        LSP= None
        
        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:

                text = page.extract_text()
                if text.startswith('ENTRY PRINT'):
                    continue
                else:

                    for line in text.split('\n'):
#                         print(line)

                        comp1 = LSP_CLASS.CTF_Job_re.search(line)
                        comp2 = LSP_CLASS.CTF_line_re.search(line)
                        Flag = LSP_CLASS.CTF_Flag_re.search(line)
                        Invoice = LSP_CLASS.CTF_Invoice_re.search(line)
                        AUPER = LSP_CLASS.CTF_AUPER_re.search(line)
                        AUSYD = LSP_CLASS.CTF_AUSYD_re.search(line)
                        AUMEL = LSP_CLASS.CTF_AUMEL_re.search(line)
                        AUBNE = LSP_CLASS.CTF_AUBNE_re.search(line)
                        AUADL = LSP_CLASS.CTF_AUADL_re.search(line)                        

                        if line.startswith('INVOICE'):
                            JOB = None
                            
                        if AUPER:
                            LSP = 'PER14'
                        if AUADL:
                            LSP = 'ADL00'
                        if AUSYD:
                            LSP = 'SYD14'
                        if AUBNE:
                            LSP = 'BNE14'
                        if AUMEL:
                            LSP = 'MEL14'
                            
                            
                        if Invoice:
                            Invoice_name = Invoice.group()                          
                        
                        if Flag:
                            Status= Status.append({'Invoice': (file + ' ' + Invoice_name), 'LSP': LSP, 'Invoice_Total': Flag.group(1)},ignore_index=True)

                        if comp1:
                            JOB = comp1.group(1).upper()
            #                 print(JOB)

                        if comp2:

                            Code = comp2.group(1)
            #                 print(Charge1)  
                            Items = comp2.group(1).split()
            #                 print(Items) 

                            pre_df = pre_df.append({'Invoice': (file + ' ' + Invoice_name), 'LSP': LSP, 'Function': 'P', 'JobNo':JOB, 'Code':Items[:-1], 'Charge':Items[-1]},ignore_index=True)
            
            pre_df['Code'] = pre_df['Code'].agg(lambda x: ''.join(map(str, x)))
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('\*')==True, None ,pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('ICS')==True, None ,pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('BALANCEDUE')==True, None ,pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TOTALAUD')==True, None ,pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('SUBTOTAL')==True, None ,pre_df['Code'])
            pre_df['Code'] = pre_df['Code'].str.upper()
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','')
            pre_df['Charge'] = pre_df['Charge'].astype(float)
            pre_df = pre_df.dropna()
            pre_df = pre_df.loc[pre_df['JobNo']!= None, :]
             
                        
            df = df.append(pre_df)
            
            
    def IND12():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        JOB = None 
       
        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()
                for line in text.split('\n'):

#                     print(line)


                    comp1 = LSP_CLASS.IND_Job_re.search(line)
                    comp3 = LSP_CLASS.IND_line_re.search(line)
                    comp4 = LSP_CLASS.IND_line2_re.search(line)
                    Flag = LSP_CLASS.IND_Flag_re.search(line)

                    if line.startswith('POD'):
                        JOB = None
                    
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': "IND12", 'Invoice_Total': Flag.group(1)},ignore_index=True)

                    if comp1:
                        JOB = comp1.group(1).upper()
#                         print(JOB)
                    
                    if comp4:
                        Charge = comp4.group(2)
                        Code = comp4.group(1)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'IND12', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge},ignore_index=True)

                    if comp3:
                        Charge = comp3.group(2)
                        Code = comp3.group(1)
                
#                         print(Code,Charge)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'IND12', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge},ignore_index=True)
                            
            Status.drop_duplicates( "Invoice" , keep='first',inplace=True)  
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df = pre_df[pre_df['Code'].str.contains('ORDER TOTAL')==False]
            pre_df=pre_df.dropna()
            pre_df['Code']=pre_df['Code'].str.rstrip('0123456789 ')
            pre_df['Code']=pre_df['Code'].str.lstrip('0123456789 ')
            pre_df['Code']=pre_df['Code'].str.replace('- ','')
            pre_df['Charge']=pre_df['Charge'].str.replace(',','')
            pre_df['Charge']=pre_df['Charge'].astype(float)


            df = df.append(pre_df)



    def DEN42():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        JOB = None 

        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()


                for line in text.split('\n'):
#                     line = line.replace(" ", "") #removes all whitespace from the line

                    Flag = LSP_CLASS.DEN_Flag_re.search(line)
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': "DEN42", 'Invoice_Total': Flag.group(1)},ignore_index=True)
#                     print(line)   

                    comp1 = LSP_CLASS.DEN_Job_re.search(line)
                    lin1 = LSP_CLASS.DEN_line1_re.search(line)
                    lin2 = LSP_CLASS.DEN_line2_re.search(line)
                    lin3 = LSP_CLASS.DEN_line3_re.search(line)         
                    lin4 = LSP_CLASS.DEN_line4_re.search(line) 
                        
                    if comp1:
                        JOB = comp1.group(1).upper()

                    if lin1:
                        Charge = lin1.group(1)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code':'BASERATE', 'Charge': Charge},ignore_index=True)

                    elif lin2:
                        Base = lin2.group(1)
                        Code = lin2.group(3)
                        Charge = lin2.group(2)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code': 'BASERATE', 'Charge': Base},ignore_index=True)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code': Code.upper(), 'Charge': Charge},ignore_index=True)

                    elif lin3:
                        Base = lin3.group(1)
                        Charge1 = lin3.group(2)
                        Code1 = lin3.group(3)
                        Charge2 = lin3.group(4)
                        Code2 = lin3.group(5)

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code':'BASERATE', 'Charge': Base},ignore_index=True)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code':Code1, 'Charge': Charge1},ignore_index=True)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code':Code2, 'Charge': Charge2},ignore_index=True)

                    elif lin4:
                        Charge = lin4.group(1)
                        Code = lin4.group(2)
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DEN42', 'Function': 'P', 'JobNo': JOB, 'Code': Code, 'Charge': Charge},ignore_index=True)
                        
                        

            pre_df.drop(pre_df.tail(1).index,inplace=True)
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Code'] = pre_df['Code'].str.replace(" ", "")
            pre_df['Charge']=pre_df['Charge'].str.replace(" ", "")
            pre_df['Charge']=pre_df['Charge'].astype(float)
            df = df.append(pre_df)



    def MXP06():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
        JOB = None 
        substring = 'Attachement to invoice'
        
        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:

        #         text = page.extract_text(x_tolerence=2,y_tolerence=0)
                text = page.extract_text()

                for line in text.split('\n'):
                    line = line.replace('.','')
                    line = line.replace(',','.')
#                     print(line)

                    if substring in line:
                        continue

                    comp1 = LSP_CLASS.MXP_Job_re.search(line)
                    comp2 = LSP_CLASS.MXP_line_re.search(line)
                    Flag = LSP_CLASS.MXP_Flag_re.search(line)

                    if comp1:
                        JOB = comp1.group(1).upper()     

                    if Flag:
                        Status= Status.append({'Invoice': (file + ' ' + JOB), 'LSP': 'MXP06', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(2)

                        Code = comp2.group(1).upper()

                        pre_df = pre_df.append({'Invoice': (file + ' ' + JOB), 'LSP': 'MXP06', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)
            
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Charge']=pre_df['Charge'].astype(float)
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('AIR FREIGHT'), 'AIR FREIGHT',pre_df['Code'])           
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TAX'), 'TAX',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('CONDITIONING'), 'CONDITIONING',pre_df['Code']) 
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TAX'), 'TAX',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('CREDO '), 'CREDO',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TAX'), 'TAX',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DELIVERY'), 'DELIVERY',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('PHARMA FEE'), 'PHARMA FEE',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DELIVERY'), 'DELIVERY',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('RECOVERY'), 'RECOVERY',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('P/U'), 'PICKUP',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DOC'), 'PAPERWORK',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DLVR'), 'DELIVERY',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DROP OFF'), 'DELIVERY',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('RECOVER'), 'PICKUP',pre_df['Code'])            
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('HANDLING'), 'HANDLING',pre_df['Code']) 
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('IN TIME KURIERSYSTEME'), 'IN TIME KURIERSYSTEME',pre_df['Code']) 
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DRY ICE'), 'DRYI',pre_df['Code']) 
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DRYICE'), 'DRYI',pre_df['Code']) 
            
            df = df.append(pre_df)

    def BUF21():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)

                    comp1 = LSP_CLASS.BUF_Job_re.search(line)
                    comp2 = LSP_CLASS.BUF_line_re.search(line)
                    Flag = LSP_CLASS.BUF_Flag_re.search(line)

                    if comp1:
                        JOB = comp1.group(1).upper()     

                    if Flag:
                        Status= Status.append({'Invoice': (file + ' ' + JOB), 'LSP': 'BUF21', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(2)

                        Code = comp2.group(1).upper()

                        pre_df = pre_df.append({'Invoice': (file + ' ' + JOB), 'LSP': 'BUF21', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)
            
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Charge'] = pre_df['Charge'].astype(float)
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('CROSS DOCK AFTER HOURS'), 'CROSS DOCK AFTER HOURS',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('CROSS DOCK TRANSFER'), 'CROSS DOCK TRANSFER',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('AFTER HOURS'), 'AFTER HOURS',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('AIR WAY BILL'), 'AIR WAY BILL',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('2ND JOBS'), 'SECOND STOP',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('AIR WAY BILL'), 'AIR WAY BILL',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DELIVERY'), 'DELIVERY',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('FUEL'), 'FUEL SURCHARGE',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('MILEAGE'), 'MILEAGE',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('PARKING'), 'PARKING',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('PICK UP AND HOLD'), 'PICK UP AND HOLD',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TENDER'), 'TENDER',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('3RD JOB'), 'SECOND STOP',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('AWB'), 'AIR WAY BILL',pre_df['Code'])
            
            df = df.append(pre_df)
                

    def DCA48():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)
                    
                    comp1 = LSP_CLASS.DCA_Job_re.search(line)
                    comp2 = LSP_CLASS.DCA_line_re.search(line)
                    comp3 = LSP_CLASS.DCA_line2_re.search(line)
                    Flag = LSP_CLASS.DCA_Flag_re.search(line)
                    
                    if comp1:
                        JOB = comp1.group(1).upper()     

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'DCA48', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(1)

                        Code = comp2.group(2).upper()

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DCA48', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)
                        
                    if comp3:
                        Charge = comp3.group(1)

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'DCA48', 'Function': 'P', 'JobNo':JOB, 'Code': 'BASE RATE (L)', 'Charge': Charge}, ignore_index=True)
                    
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','') 
            pre_df['Charge'] = pre_df['Charge'].astype(float)
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('BASE RATE'), 'BASE RATE (L)',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('OVERWEIGHT'), 'OVERWEIGHT',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('HANDLING'), 'HANDLING',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('OVER BASE MILES'), 'OVER BASE MILES',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('ATTEMPT'), 'ATTEMPT',pre_df['Code'])  
            df = df.append(pre_df)        
            
            
            

    def SEL07():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)
                    
                    comp1 = LSP_CLASS.SEL_Job_re.search(line)
                    comp2 = LSP_CLASS.SEL_line_re.search(line)
                    Flag = LSP_CLASS.SEL_Flag_re.search(line)

                    if comp1:
                        JOB = comp1.group(1).upper()  
                        pre_df = pre_df.append({'Invoice': (file + ' ' + JOB), 'LSP': 'SEL07', 'Function': 'P', 'JobNo':JOB, 'Code':None, 'Charge':None}, ignore_index=True)
                        Status= Status.append({'Invoice': (file + ' ' + JOB), 'LSP': 'SEL07', 'Invoice_Total': Invoice_Total}, ignore_index=True)
                        
                    if Flag:
                        Invoice_Total = Flag.group(1)
                                                
                        
                    if comp2:
                        Charge = comp2.group(2)

                        Code = comp2.group(1).upper()

                        pre_df = pre_df.append({'Invoice': None, 'LSP': 'SEL07', 'Function': 'P', 'JobNo':None, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)
            
            pre_df['Invoice'] = pre_df['Invoice'].fillna(method="bfill")
            pre_df['JobNo'] = pre_df['JobNo'].fillna(method="bfill")
            pre_df = pre_df.dropna()
            pre_df = pre_df[pre_df['Code'].str.contains('SUB')==False]
            pre_df = pre_df[pre_df['Code'].str.contains('REF')==False]
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','') 
            pre_df['Charge'] = pre_df['Charge'].astype(float, errors = 'raise')
            
            df = df.append(pre_df)
            
            
    def HNL09():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)
                    
                    comp1 = LSP_CLASS.HNL_Job_re.search(line)
                    comp2 = LSP_CLASS.HNL_line_re.search(line)
                    comp3 = LSP_CLASS.HNL_line2_re.search(line)
                    Flag = LSP_CLASS.HNL_Flag_re.search(line)
                    
                    if comp1:
                        JOB = comp1.group(1).upper()
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'HNL09', 'Function': 'P', 'JobNo':JOB, 'Code':None, 'Charge':None}, ignore_index=True)

                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'HNL09', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(1)
                        Code = comp2.group(2).upper()
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'HNL09', 'Function': 'P', 'JobNo':None, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)
                        
                    if comp3:
                        Charge = comp3.group(1)

                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'HNL09', 'Function': 'P', 'JobNo':None, 'Code': 'BASE RATE (L)', 'Charge': Charge}, ignore_index=True)
                    
            
            pre_df['JobNo'] = pre_df['JobNo'].fillna(method="bfill")
            pre_df = pre_df.dropna()
            pre_df['Code'] = pre_df['Code'].astype(str)
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','') 
            pre_df['Charge'] = pre_df['Charge'].astype(float)
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('BASE RATE'), 'BASE RATE (L)',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('OVERWEIGHT'), 'OVERWEIGHT',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('HANDLING'), 'HANDLING',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('OVER BASE MILES'), 'OVER BASE MILES',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('ATTEMPT'), 'ATTEMPT',pre_df['Code'])  
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('HOT RUSH'), 'HOT RUSH',pre_df['Code']) 
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('TAX'), 'TAX',pre_df['Code'])             
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DIM VS ACTUAL'), 'DIM VS ACTUAL',pre_df['Code'])             
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('FSC'), 'FSC',pre_df['Code'])             
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('ADD MOVE'), 'ADD MOVE',pre_df['Code'])     
            
            df = df.append(pre_df) 
            
    def ZRH00():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file,) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
#                     print(line)
                    
                    comp1 = LSP_CLASS.ZRH_Job_re.search(line)
                    comp2 = LSP_CLASS.ZRH_line_re.search(line)
                    Flag = LSP_CLASS.ZRH_Flag_re.search(line)           
                    
                    if comp1:
                        JOB = comp1.group(1).upper()
                        
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'ZRH00', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(2)
                        Code = comp2.group(1).upper()
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'ZRH00', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)    
                        
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('FUEL SURCHARGE'), 'FUEL SURCHARGE',pre_df['Code'])              
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('DRYICE'), 'DRYI',pre_df['Code'])
            pre_df['Code'] = np.where(pre_df['Code'].str.contains('ADD MOVE'), 'ADD MOVE',pre_df['Code'])
            
            df = df.append(pre_df)                         
                        
                        
    def AMS14():
        global Status
        global lines 
        global df
        pre_df =  pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})

        with pdfplumber.open(file) as pdf:
            pages = pdf.pages
            for page in pdf.pages:
                text = page.extract_text()

                for line in text.split('\n'):
                    print(line)
                    
                    comp1 = LSP_CLASS.AMS_Job_re.search(line)
                    comp2 = LSP_CLASS.AMS_line_re.search(line)
                    Flag = LSP_CLASS.AMS_Flag_re.search(line)           
                    
                    if comp1:
                        JOB = comp1.group(1).upper()
                        
                    if Flag:
                        Status= Status.append({'Invoice': file, 'LSP': 'AMS14', 'Invoice_Total': Flag.group(1)}, ignore_index=True)
                        
                    if comp2:
                        Charge = comp2.group(2)
                        Code = comp2.group(1).upper()
                        pre_df = pre_df.append({'Invoice': file, 'LSP': 'AMS14', 'Function': 'P', 'JobNo':JOB, 'Code':Code.upper(), 'Charge':Charge}, ignore_index=True)    

            pre_df = pre_df[pre_df['Code'].str.contains('TOTAL')==False]
            pre_df = pre_df[pre_df['Code'].str.contains('VAT 0% OF')==False]
            pre_df['Charge'] = pre_df['Charge'].str.replace(',','') 
            
            
            
            df = df.append(pre_df)    
                        
                        

In [68]:
x

'invoice_523975.pdf'

Main Algo

In [152]:
%%time
lines = []

Status = pd.DataFrame({'Invoice': [], 'LSP': [], 'Invoice_Total': []})
df = pd.DataFrame({'Invoice': [], 'LSP': [], 'Function': [], 'JobNo':[], 'Code':[], 'Charge':[]})
x = [a for a in os.listdir('.') if a.lower().endswith(".pdf")]

for b in x:

    file = b
    
    with pdfplumber.open(file) as pdf:
        pages = pdf.pages
        for page in pdf.pages:
            text = page.extract_text()

            for line in text.split('\n'):
                LSP_Search = LSP_CLASS.LSP_re.search(line)
                
                if LSP_Search:
                    LSP = LSP_CLASS.DICT.get(LSP_Search.group(),'NOT FOUND')
                    print(file,LSP)
                    
                    if LSP == 'MAD07':
                        LSP_CLASS.MAD07()                                    

                    if LSP == 'SEA70':
                        LSP_CLASS.SEA70()
                                       
                    if LSP == 'FRA06':   
                        LSP_CLASS.FRA06()
                
                    if LSP == 'BRU09':   
                        LSP_CLASS.BRU09()
                        
                    if LSP == 'IAD58':   
                        LSP_CLASS.IAD58()
                        
                    if LSP == 'CVG02':   
                        LSP_CLASS.CVG02()
                        
                    if LSP == 'YYZ37':   
                        LSP_CLASS.YYZ37()
                        LSP_CLASS.YYZ_df_cleanup(df)     
                        
                    if LSP == 'CTF':
                        LSP_CLASS.CT_Freight()
                        
                    if LSP == 'IND12':
                        LSP_CLASS.IND12()
                        
                    if LSP == 'DEN42':
                        LSP_CLASS.DEN42()
                  
                    if LSP == 'MXP06':
                        LSP_CLASS.MXP06()
                        
                    if LSP == 'BUF21':
                        LSP_CLASS.BUF21()
                        
                    if LSP == 'DCA48':
                        LSP_CLASS.DCA48()
                        
                    if LSP == 'SEL07':
                        LSP_CLASS.SEL07()                  
                                            
                    if LSP == 'HNL09':
                        LSP_CLASS.HNL09()
                        
                    if LSP == 'ZRH00':
                        LSP_CLASS.ZRH00()
                        
                    if LSP == 'AMS14':
                        LSP_CLASS.AMS14()


                    break
                    
            break
            

df.to_excel('DF_TESTING.xlsx',index= False)  
df
            
   

Quick 1000771.pdf SEA70
Quick 1000776.pdf SEA70
Sterling 1000774.pdf SEA70
CPU times: user 5.76 s, sys: 24.6 ms, total: 5.79 s
Wall time: 5.79 s


,Invoice,LSP,Function,JobNo,Code,Charge
0,Quick 1000771.pdf,SEA70,P,199850340W,BASE RATE,70.00
1,Quick 1000771.pdf,SEA70,P,199850340W,EXCESS WGT,3.10
2,Quick 1000771.pdf,SEA70,P,199850340W,MILIAGE CHARGE,19.00
4,Quick 1000771.pdf,SEA70,P,199850340W,WEEKEND PU,15.00
5,Quick 1000771.pdf,SEA70,P,199850340W,FUEL SURCHARGE,13.35
...,...,...,...,...,...,...
5,Sterling 1000774.pdf,SEA70,P,199950444M,ADDITIONAL AIRBILL,10.00
6,Sterling 1000774.pdf,SEA70,P,199950444M,BASE RATE,33.50
7,Sterling 1000774.pdf,SEA70,P,199950444M,DELIVERY WAITING,7.50
9,Sterling 1000774.pdf,SEA70,P,199950444M,WEEKEND DELIVERY,15.00


TESTING 

Data Clean Up + Mapping To QT_Code

In [153]:
%%time
#https://docs.google.com/spreadsheets/d/1YB9qtHEn08-r-MnXWQ0iy-pgumyKXvQv7lzxqb9kpIA/edit#gid=0

pd.options.display.float_format = "{:,.2f}".format


# #create a data frame dictionary to store your data frames
# DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}


# for key in DataFrameDict.keys():
#     DataFrameDict[key] = df[:][df.LSP == key]
#     exec(f'LSP_{key} = DataFrameDict[key]')

df['LSP'] = df['LSP'].fillna(method="bfill")
df['JobNo'] = df['JobNo'].fillna(method="bfill")
df = df.dropna()

#create unique list of names
UniqueNames = df.LSP.unique()

for i in UniqueNames:


    if i == 'BRU09':
        df.loc[df['LSP']== 'BRU09',:]  = df.loc[(df['Charge']!='0,00') & (df['LSP']== 'BRU09'),:]
        df.loc[df['LSP']== 'BRU09', 'JobNo'] = df.loc[df['LSP']== 'BRU09', 'JobNo'].str.lstrip('0')
        df.loc[df['LSP']== 'BRU09', 'Charge'] = df.loc[df['LSP']== 'BRU09', 'Charge'].str.replace(',', '.')
        df.loc[df['LSP']== 'BRU09', 'Charge'] = df.loc[df['LSP']== 'BRU09', 'Charge'].astype(float, errors = 'raise')
        df.loc[df['LSP']== 'BRU09', 'Code'] = df.loc[df['LSP']== 'BRU09', 'Code'].str.split(',').str[0]
        df.loc[df['LSP']== 'BRU09', 'Code'] = df.loc[df['LSP']== 'BRU09', 'Code'].astype(str)
        df.loc[df['LSP']== 'BRU09', 'Code'] = df.loc[df['LSP']== 'BRU09', 'Code'].str.rstrip('0123456789 ')
        df.loc[df['LSP']== 'BRU09', 'Code'] = df.loc[df['LSP']== 'BRU09', 'Code'].str.upper() 
        df['Code'] = np.where(df['Code'].str.contains('NEXT DROP'), 'NEXT DROP', df['Code'])
        
    
    if i == 'YYZ37': 
        df.loc[df['LSP']== 'YYZ37', 'Code'] = np.where(df.loc[df['LSP']== 'YYZ37', 'Code'].str.contains('\$')==True, None,df.loc[df['LSP']== 'YYZ37', 'Code'] )
        
        
        
        
           
        
        
df['Code'] = np.where(df['Code'].str.contains('STORAGE'), 'WAREHOUSE',df['Code'])  
df['Code'] = np.where(df['Code'].str.contains('CLEARANCE'), 'CLEARANCE',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('ICE'), 'DRYICE',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('RETURN'), 'RETURN',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('COLLECTION'), 'COLLECTION',df['Code'])               
df['Code'] = np.where(df['Code'].str.contains('SEA'), 'TRANSFER OFFICE TO SEA',df['Code'])  
df['Code'] = np.where(df['Code'].str.contains("X'S"), 'PICK UP SUPPLIES',df['Code'])  
df['Code'] = np.where(df['Code'].str.contains('TOLL'), 'TOLLS',df['Code'])  
df['Code'] = np.where(df['Code'].str.contains('PAPERWORK'), 'PAPERWORK',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('WAITING'), 'WAITING TIME',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('ATTEMPT'), 'ATTEMPT',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('HOLIDAY'), 'HOLIDAY',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('SUPPLY'), 'SUPPLY',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('SUPPLIES'), 'SUPPLY',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('GPS'), 'GPS',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains(r'PU \d'), 'PICK UP',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('STOP'), 'STOP',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('CLEARANCE'), 'CLEARANCE',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('RETURN'), 'RETURN',df['Code']) 
df['Code'] = np.where(df['Code'].str.contains('COLLECTION'), 'COLLECTION',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('SPECIAL'), 'SPECIAL',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('VET'), 'VET',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('COVID'), 'COVID',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('WEEKEND'), 'WEEKEND',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('TRANSFER'), 'TRANSFER',df['Code'])
df['Code'] = np.where(df['Code'].str.contains('AFTER HOURS'), 'AFTER HOURS',df['Code'])


df = df.dropna()
# df.to_excel('DF_TESTING.xlsx',index= False)


# #---------------------------------------------------------------------------------------------------------------

def get_code(code):
    return qcodes_dict.get(code,'Not Found')

sa = gspread.service_account(filename='quick-spread-sheets-b0869931c50e.json')
sh = sa.open('Quick LSP Conversion Codes')
wks = sh.worksheet("Sheet1")

Sheet = wks.get_all_records()
qtrac_codes = pd.DataFrame(Sheet)
qtrac_codes.set_index('Code') 
qcodes_dict = dict(zip(qtrac_codes['Code'],qtrac_codes['QT_CODE']))
df['QT_CODE']=df['Code'].map(get_code)

df_NOTFOUND = df[df["QT_CODE"].str.contains("Not Found") == True]
pv = pd.pivot_table(df_NOTFOUND, values='Charge', index=['Code','LSP','Invoice'], aggfunc=np.count_nonzero)
pv.to_csv('LSP_to_QT_Missing_Code.csv', index= True)
df['Charge'] =  df['Charge'].astype(float, errors = 'raise')
df




CPU times: user 197 ms, sys: 15.6 ms, total: 213 ms
Wall time: 1.45 s


,Invoice,LSP,Function,JobNo,Code,Charge,QT_CODE
0,Quick 1000771.pdf,SEA70,P,199850340W,BASE RATE,70.00,L
1,Quick 1000771.pdf,SEA70,P,199850340W,EXCESS WGT,3.10,W
2,Quick 1000771.pdf,SEA70,P,199850340W,MILIAGE CHARGE,19.00,D
4,Quick 1000771.pdf,SEA70,P,199850340W,WEEKEND,15.00,WKC
5,Quick 1000771.pdf,SEA70,P,199850340W,FUEL SURCHARGE,13.35,FS
...,...,...,...,...,...,...,...
5,Sterling 1000774.pdf,SEA70,P,199950444M,ADDITIONAL AIRBILL,10.00,AWB
6,Sterling 1000774.pdf,SEA70,P,199950444M,BASE RATE,33.50,L
7,Sterling 1000774.pdf,SEA70,P,199950444M,WAITING TIME,7.50,WT
9,Sterling 1000774.pdf,SEA70,P,199950444M,WEEKEND,15.00,WKC


In [154]:
#https://docs.google.com/spreadsheets/d/1YB9qtHEn08-r-MnXWQ0iy-pgumyKXvQv7lzxqb9kpIA/edit#gid=0
pv

,,
Code,LSP,Invoice


In [101]:
Status
    

,Invoice,LSP,Invoice_Total
0,invoice_523975.pdf,AMS14,"5,654.00"


In [155]:
History = pd.read_csv('History.csv',encoding= 'unicode_escape',low_memory=False)
pd.set_option('max_colwidth', 400)

df['Charge'] =  df['Charge'].astype(float, errors = 'raise')
Pricetable = pd.pivot_table(df, values='Charge', index=['Invoice'], aggfunc=np.sum)
Status_Report = pd.merge(Status, Pricetable, on ='Invoice', how ='inner')

Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'].str.replace(' EUR', '')
Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'].str.replace('Total Amount ', '')
Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='FRA06','Invoice_Total'].str.replace('Total amount ', '')
Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'].str.replace(' Euros', '')
Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'].str.replace('.','')
Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='MAD07','Invoice_Total'].str.replace(',','.')
Status_Report.loc[Status_Report['LSP']=='SEA70','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='SEA70','Invoice_Total'].str.rstrip(' USD')
Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'].str.lstrip('TOTAL TO PAY (VAT Included): ')
Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'].str.rstrip(' EUR')
Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'] = Status_Report.loc[Status_Report['LSP']=='BRU09','Invoice_Total'].str.replace(',','.')
Status_Report.loc[:,'Invoice_Total'] = Status_Report.loc[:,'Invoice_Total'].str.replace(',','')
Status_Report.rename(columns={'Charge':"Summed_Charges"},inplace=True)
Status_Report = Status_Report.assign(Previously_Converted=Status_Report.Invoice.isin(History.Invoice).astype(bool))
Status_Report.drop_duplicates( "Invoice" , keep='first',inplace=True)
# Status_Report = np.where(Status_Report['Invoice_Total'] == Status_Report['Summed_Charges'], Status_Report['Flag'] == True ,Status_Report['Flag'] == False)   

try:
    Status_Report.loc[:,'Invoice_Total'] = Status_Report.loc[:,'Invoice_Total'].astype(float)
except:
    print('did not adjust Invoice_Total Float')

    
Status_Report['Flag'] = np.where(Status_Report['Invoice_Total'] == Status_Report['Summed_Charges'], True, False )
    

    
Status_Report


/Users/gregdilella/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':


,Invoice,LSP,Invoice_Total,Summed_Charges,Previously_Converted,Flag
0,Quick 1000771.pdf,SEA70,"14,850.78","14,850.78",False,True
1,Quick 1000776.pdf,SEA70,"5,212.44","5,212.44",False,True
2,Sterling 1000774.pdf,SEA70,287.85,287.85,False,True


In [156]:

LSP_list_Tableau.rename(columns = {'Job No':'JobNo','Function Code':'QT_Function_Code','Vendor':'QT_LSP'}, inplace = True)
df['JOBLSP'] = df['JobNo']+df['LSP']
LSP_list_Tableau['JOBLSP'] = LSP_list_Tableau['JobNo'] + LSP_list_Tableau['QT_LSP']
LSP_list_Tableau = LSP_list_Tableau.drop_duplicates(subset=['JOBLSP'], keep='first')

df = pd.merge(df, LSP_list_Tableau, on ='JOBLSP', how ='left', suffixes=('_invoice','_Qtrac'))

df['Function'] = np.where(df['QT_Function_Code'].isnull(), df['Function'] ,df['QT_Function_Code']) 
df['Dup'] = df.duplicated(subset=['JobNo_invoice', 'Code' ,'Charge'])
# df.to_csv('inspect.csv', index= False)
df

,Invoice,LSP,Function,JobNo_invoice,Code,Charge,QT_CODE,JOBLSP,QT_Function_Code,JobNo_Qtrac,QT_LSP,Dup
0,Quick 1000771.pdf,SEA70,P,199850340W,BASE RATE,70.00,L,199850340WSEA70,P,199850340W,SEA70,False
1,Quick 1000771.pdf,SEA70,P,199850340W,EXCESS WGT,3.10,W,199850340WSEA70,P,199850340W,SEA70,False
2,Quick 1000771.pdf,SEA70,P,199850340W,MILIAGE CHARGE,19.00,D,199850340WSEA70,P,199850340W,SEA70,False
3,Quick 1000771.pdf,SEA70,P,199850340W,WEEKEND,15.00,WKC,199850340WSEA70,P,199850340W,SEA70,False
4,Quick 1000771.pdf,SEA70,P,199850340W,FUEL SURCHARGE,13.35,FS,199850340WSEA70,P,199850340W,SEA70,False
...,...,...,...,...,...,...,...,...,...,...,...,...
787,Sterling 1000774.pdf,SEA70,D,199950444M,ADDITIONAL AIRBILL,10.00,AWB,199950444MSEA70,D,199950444M,SEA70,False
788,Sterling 1000774.pdf,SEA70,D,199950444M,BASE RATE,33.50,L,199950444MSEA70,D,199950444M,SEA70,False
789,Sterling 1000774.pdf,SEA70,D,199950444M,WAITING TIME,7.50,WT,199950444MSEA70,D,199950444M,SEA70,False
790,Sterling 1000774.pdf,SEA70,D,199950444M,WEEKEND,15.00,WKC,199950444MSEA70,D,199950444M,SEA70,False


Transformation and Output

In [157]:
df.rename(columns={"LSP": "LSP Code","Function": "LSP Function","JobNo_invoice": "Job Number"},inplace=True)
table = pd.pivot_table(df, values='Charge', index=['Job Number','LSP Code','LSP Function'], columns=['QT_CODE'], aggfunc=np.sum)


with pd.ExcelWriter("Conversion.xlsx", engine = 'xlsxwriter') as writer:

    df.to_excel(writer, sheet_name='Invoice_Conversion',index=False)
    table.to_excel(writer, sheet_name='QT_upload',float_format="%.2f")
    Status_Report.to_excel(writer, sheet_name='Invoice_Status_Report',index=False)
    df_NOTFOUND.to_excel(writer, sheet_name='Charges_Need_QTCode',index=True)
    
    workbook  = writer.book
    worksheet = writer.sheets['QT_upload']
    format1 = workbook.add_format({'num_format': '0.00'})
    worksheet.set_column('D:AA', None, format1)  
    worksheet2 = writer.sheets['Invoice_Conversion']
    worksheet2.set_column('F:F', None, format1)  
    
    


   

In [158]:
History = History.append(Status_Report)
History = History.drop(columns =['Previously_Converted'])
History = History.drop_duplicates(subset=['Invoice'], keep='first')
History.to_csv('History.csv',index=False)

In [54]:
file = ""
with pdf = pdfplumber.open(file) as pdf:
    pages = pdf.pages
        for page in pdf.pages:
        text = page.extract_table()
gh = pd.DataFrame(text)
gh


""


In [69]:
file = 'invoice_523975.pdf'

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()

        for line in text.split('\n'):
            print(line)

Collective Invoice
QuickSTAT JFK NY
Attn: accounting
175-28 148TH AVENUE
JAMAICA, NY 11434
United States
Invoice Number: 523975
Invoice Date: 7 Sep 2022
Invoice Due Date: 21 Sep 2022
Customer Number: 10301
Page: 1 of 5
We hereby charge you for having your delivery transported.
Shipment No: AMS-AE-014553 Your Reference: 199720104W
AWB: 074-55738465
Quick special pickup 66.00 EUR
Quick handling EU/ Non-EU 70.00 EUR
Quick forwarding charge 318.12 EUR
Quick fuel charge 12.00 EUR
Quick security surcharges 10.00 EUR
Quick security check 48.00 EUR
Quick Drive fuel charge 12% 7.92 EUR
Quick Dangerous Goods Fee 70.00 EUR
Quick Dry ice fee 46.00 EUR
Total / Shipment: 648.04 EUR
Shipment No: AMS-AE-014554 Your Reference: 197570510MR
AWB: 074-55738524
Quick special pickup 49.50 EUR
Quick handling EU/ Non-EU 70.00 EUR
Quick forwarding charge 1,285.20 EUR
Quick fuel charge 89.00 EUR
Quick security surcharges 16.82 EUR
Quick security check 48.00 EUR
Quick Drive fuel charge 12% 5.94 EUR
Total / Shipme